### 아이디어
- 버블소트는 n번째 loop에서 n번째로 큰 숫자가 맨 오른쪽으로 이동하여 정렬에서 제외된다.
- 세그먼트 트리를 구성한 후, 세그트리에서 가장 큰 숫자를 제거할 때 마다 자신보다 오른쪽에 있는 숫자들의 개수를 증가시킨다.
  - 가장 큰 숫자가 여러개일 경우 -> 애초에 가장 큰 숫자를 고를 때 가장 오른쪽에 있는 숫자를 고른다.
    - 이를 구현하기 위해 세그 트리에 값을 저장할 때 `(값, index)`의 형태로 넣고, 가장 큰 숫자를 고를때 key를 `값, index` (높은거 우선)순서로 뽑는다. 
    - `max()` 는 $O(logN)$이므로 $O(NlogN)$만에 구한다.
    - 최대 값의 인덱스를 찾은 뒤, default값이 처음 등장하는 위치까지를 `answer`에 더한다. `l.find()` = $O(N)$ 이지만 세그트리의 특성상 리프 노드의 개수가 $\frac {1}{2}$ 지점 이후에 있으므로 $O(N * N^{1 \over 2}) = O(N^{2 \over 3})$. 
    - 느리다. N이 50만이여서 $O(NlogN)$만에 풀어야 하고, 세그트리랑 관련도 없어보인다.

### 풀이
- (원리) 어떤 수 `a`, `b`가 있을 때, `a` < `b`이면서 `b`보다 오른쪽에 있는 수의 개수를 구하는 것은 `out-of-order 한 숫자의 개수`를 구하는 것과 같다. 
  - 이를 `Inversion Counting`이라고 하는 것 같다.
  - 버블 정렬의 `swap`은 인접한 수에서 `out-of-order`한 것을 발견했을 때마다 수행한다.
  - loop가 돌면 원소 1개의 위치가 결정되므로 정렬할 원소를 1개 빼는 것과 같다.
  - 수열에서 out-of-order 한 쌍이 없어질 때까지 반복한다.
- (구현) 입력으로 주워진 수열 `L`을 정렬한 `L[i]`의 `i` 저장하는 배열 `L2`를 만든다.
  - 즉, 정렬에 의해 인덱스가 섞이는 형태가 될 것이다.
  - 임의의 쌍 `i, j`에 대해 L[i] < L[j] 이면서 L2[i] > L2[j]인 경우를 찾는다.

- 아니, 설명이 너무 어렵다. 코드의 주석으로 설명하는게 더 쉬울 것 같다.

In [ ]:
import io, os
input = io.BytesIO(os.read(0, os.fstat(0).st_size)).readline

class SegmentTree:
  def __init__(self, L, default, f):
    self._def = default
    self._f = f
    self._len = len(L)
    self._size = _size = 1 << (self._len - 1).bit_length()

    self.L = [default] * (2 * _size)
    self.L[_size:_size + self._len] = L
    for i in reversed(range(_size)):
      self.L[i] = f(self.L[i + i], self.L[i + i + 1])

  def __delitem__(self, idx):
    self[idx] = self._def

  def __getitem__(self, idx):
    return self.L[idx + self._size]

  def __setitem__(self, idx, value):
    idx += self._size
    self.L[idx] = value
    idx >>= 1
    while idx:
      self.L[idx] = self._f(self.L[2 * idx], self.L[2 * idx + 1])
      idx >>= 1

  def __len__(self):
    return self._len

  def query(self, s, e):
    s += self._size
    e += self._size

    l = r = self._def
    while s < e: 
      if s & 1:
        l = self._f(l, self.L[s])
        s += 1
      if e & 1:
        e -= 1
        r = self._f(self.L[e], r)
      s >>= 1
      e >>= 1

    return self._f(l, r)

N = int(input())
L = [*map(int, input().split())]
L2 = [*range(N)]
L2.sort(key=lambda x: L[x])

seg = SegmentTree([0] * N, 0, lambda x, y: x + y) #순서대로 배열된 원소의 개수
answer = 0
for idx in L2: #섞여있는 원소의 idx(임의의 원소라고 생각해도 좋다)
  answer += seg.query(idx, N) #inversion이 발생한 횟수. bubble sort로 생각한다면, 원소의 idx가 순서대로 정렬된 원소를 거쳐 지나갔으니까, 이 쿼리의 값과 같은 횟수만큼 swap한 것이라고 볼 수 있다.
  seg[idx] = 1 #이 원소가 정렬 됐음을 표시한다. bubble sort의 경우 한번의 loop마다 1개의 원소가 정렬되므로 loop 한번을 돌았다고 봐도된다.

print(answer)